In [1]:
# Import libraries
import matplotlib.pyplot as plt
import math
import nltk
import numpy as np
import pandas as pd
import re
import seaborn as sns
import string

from collections import Counter
from numpy import sqrt 
from tqdm import tqdm

from sklearn import metrics
from sklearn.metrics import mean_squared_error

from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors

from scipy.sparse import csr_matrix

In [2]:
from google.colab import drive

Define Data

In [3]:
# Connect to GDrive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
dataset = pd.read_csv('/content/drive/MyDrive/KULIAH/TA-2/SIDANG/TASI-107/DATA/Data_Preprocessing.csv')
dataset.sample(3)

,reviewerId,placeId,destination_name,description,category_name,text,publishedAtDate,user_rating,overview
1016,957,31,Pakkodian Toba,Selanjutnya melangkah ke pantai pakkodian yang...,wisata bahari,.,2020-12-15T05:44:41.674Z,5,wisata bahari lang pantai pakkodian camat tamp...
4548,462,57,Pantai Lumban Bul-bul Balige,Tempat wisata di Toba Samosir yang kedua adala...,wisata bahari,Pantai wisata danau toba dengan pasir putih ya...,2019-01-02T16:56:15.596Z,5,wisata bahari wisata toba samosir wisata panta...
2667,2388,48,Air Terjun Situmurun,Air terjun ini sangat indah dan asri. Pemandan...,wisata alam,"Air terjun yang langsung jatuh ke Danau Toba, ...",2018-04-04T10:49:55.367Z,5,wisata alam air terjun indah asri pandang air ...


#CBF


In [5]:
feature =dataset[['placeId','overview']]
x =feature
y =dataset['user_rating']

In [6]:
# split the data set into train and test
x_train, x_test, y_train, y_test =train_test_split(x, y, test_size=0.1, random_state=4, stratify=y)

In [7]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(9584, 2)
(9584,)
(1065, 2)
(1065,)


In [8]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
# instantiate the vectorizer object
tf_idf_vect = TfidfVectorizer(ngram_range=(1,2))

# Fitting and transforming the vector
train_tf_idf = tf_idf_vect.fit_transform(x_train['overview'])

#test data
test_tfidf=tf_idf_vect.transform(x_test['overview'])

In [9]:
# instantiate learning model (k = 20)
model_cbf = KNeighborsClassifier(metric='cosine', n_neighbors= 20)

# fitting the model on TfidfVectorizer train
model_cbf.fit(train_tf_idf, y_train)

# predict the response on the TfidfVectorizer train
pred_tfidf = model_cbf.predict(test_tfidf)

# evaluate
mse = mean_squared_error(y_test, pred_tfidf)
rmse_cbf = np.sqrt(mse)
print("RMSE: ", rmse_cbf)

RMSE:  1.1514432666198242


# CF

In [10]:
CF=dataset[['placeId','reviewerId', 'user_rating']]
CF.head()

,placeId,reviewerId,user_rating
0,1,0,5
1,1,1,5
2,1,2,5
3,1,3,5
4,2,4,5


In [12]:
#90% baris data dipilih secara random sebagai dara train
CF_train = CF.sample(frac = 0.9, random_state = 1)
CF_train = CF_train.reset_index(drop=True)
CF_test = CF.drop(CF_train.index)
CF_test = CF_test.reset_index(drop=True)

In [13]:
x_train = CF_train[['reviewerId','placeId']]
y_train = CF_train[['user_rating']]
x_test = CF_test[['reviewerId','placeId']]
y_test = CF_test[['user_rating']]

In [14]:
x_train

,reviewerId,placeId
0,6595,58
1,4917,57
2,2959,53
3,487,32
4,4272,57
...,...,...
9579,6126,58
9580,7284,59
9581,7343,59
9582,3244,57


In [15]:
x_test

,reviewerId,placeId
0,7990,60
1,7991,60
2,749,60
3,7992,60
4,7993,60
...,...,...
1060,8747,65
1061,2679,65
1062,8748,65
1063,8749,65


In [16]:
# the number of users
n_users = x_train.reviewerId.unique().shape[0]

# the number of items in the dataset
n_items = x_train.placeId.unique().shape[0]

#max reviewerId
n_users = x_train['reviewerId'].max()

#max placeId
n_items = x_train['placeId'].max()

#initialize empty array
A = np.zeros((n_users,n_items))


In [17]:
# mengubah kumpulan data menjadi kumpulan data biner

for i in range (len(A)):
  for j in range(len(A[0])):
    if A[i][j]>=3:
      A[i][j]=1
    else:
      A[i][j] = 0



In [18]:
matrix = CF.pivot(index='reviewerId',columns='placeId',values='user_rating').fillna(0)

display(matrix.head())

placeId,1,2,3,4,5,6,7,8,9,10,...,56,57,58,59,60,61,62,63,64,65
reviewerId,,,,,,,,,,,,,,,,,,,,,
0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
3,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0
4,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
print("Original rating materix :  ",A)

Original rating materix :   [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [20]:
#konversi matrix untuk menghemat memori
sparse_matrix = csr_matrix(A)

###Mengisi data sparse dengan rata-rata rating


In [21]:
import statistics  

In [22]:
def calculate_rating(list_item_id, CF):
  list_rating_tmp = []

  for i in list_item_id:
    avg_rating = CF[CF["reviewerId"] == i][["user_rating"]].mean()
    list_rating_tmp.append(float(avg_rating))

    return round(statistics.mean(list_rating_tmp), 2)
  
def predict_nearest_item(reviewerId, list_item, sparse_matrix, model_knn, k):
  list_index = []

  for i in list_item:
    distances, indices = model_knn.kneighbors(sparse_matrix[i],
                                               n_neighbors=k)
    indices = indices.flatten()
    indices = indices[1:]
    list_index.extend(indices)

  return list_index

def knn_predict(df_test, sparse_matrix, model_knn, k):
  list_rating_tmp = []
  for i in df_test['reviewerId']:
    filter_user = df_test[df_test["reviewerId"] == i].placeId
    filter_user = filter_user.tolist()
    list_item = predict_nearest_item(i, filter_user, sparse_matrix, model_knn, k)
    list_rating_tmp.append(calculate_rating(list_item, CF))
  return np.array(list_rating_tmp)

In [23]:
def eksperimen_hiperparameter_knn(matriks_rating, x_uji, y_uji, k):
  knn = NearestNeighbors(n_neighbors=k, algorithm='brute', p = 2)
  knn.fit(matriks_rating)
  pred2 = knn_predict(x_uji, matriks_rating, knn, k)
  rmse = np.sqrt(np.mean((pred2 - y_uji.to_numpy())**2))
  return rmse

In [24]:
print("matriks_rating")

matriks_rating


In [25]:
list_k = [3, 5, 10, 20]
list_rmse_nonsparse = []
for i in list_k:
  rmse_val = eksperimen_hiperparameter_knn(sparse_matrix, x_test, y_test, i)
  list_rmse_nonsparse.append(rmse_val)
  print("nilai k =", i, "RMSE = %.4f" %rmse_val)

nilai k = 3 RMSE = 1.0963
nilai k = 5 RMSE = 1.0760
nilai k = 10 RMSE = 1.0760
nilai k = 20 RMSE = 1.0963


In [26]:
list_rmse_nonsparse

[1.0963019357240507,
 1.0759862819168464,
 1.0759862819168464,
 1.0963019357240507]

# **HYBRID**

In [27]:
# Displaying the test results from training the Hybrid Recommender on test data
hybrid_predictions_test = (0.3 * (rmse_cbf) + 0.7 * min(list_rmse_nonsparse))

print("RMSE hybrid recommendations (test data): {}".format(hybrid_predictions_test))

RMSE hybrid recommendations (test data): 1.0986233773277396


In [28]:
# Displaying the test results from training the Hybrid Recommender on test data
hybrid_predictions_test = ( 0.4 * (rmse_cbf) + 0.6 * min(list_rmse_nonsparse))

print("RMSE hybrid recommendations (test data): {}".format(hybrid_predictions_test))

RMSE hybrid recommendations (test data): 1.1061690757980376


In [29]:
# Displaying the test results from training the Hybrid Recommender on test data
hybrid_predictions_test = ( 0.5 * (rmse_cbf) + 0.5 * min(list_rmse_nonsparse))

print("RMSE hybrid recommendations (test data): {}".format(hybrid_predictions_test))

RMSE hybrid recommendations (test data): 1.1137147742683353


In [30]:
# Displaying the test results from training the Hybrid Recommender on test data
hybrid_predictions_test = ( 0.6 * (rmse_cbf) + 0.4 * min(list_rmse_nonsparse))

print("RMSE hybrid recommendations (test data): {}".format(hybrid_predictions_test))

RMSE hybrid recommendations (test data): 1.121260472738633


In [31]:
# Displaying the test results from training the Hybrid Recommender on test data
hybrid_predictions_test = ( 0.7 * (rmse_cbf) + 0.3 * min(list_rmse_nonsparse))

print("RMSE hybrid recommendations (test data): {}".format(hybrid_predictions_test))

RMSE hybrid recommendations (test data): 1.1288061712089308
